<a href="https://colab.research.google.com/github/Arya1790/Deep-Learning/blob/main/LSTM_next_word__prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Next Word Prediction Using LSTM

We use the text of Shakespeare's "Hamlet" as our dataset. This rich, complex text provides a good challenge for our model.

In [ ]:
## Data Collection
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import  pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

## load the dataset
data=gutenberg.raw('shakespeare-hamlet.txt')
## save to a file
with open('hamlet.txt','w') as file:
    file.write(data)

with open('hamlet.txt','r') as file:
    text=file.read().lower()

## Tokenize the text-creating indexes for words
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
total_words

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


4818

In [ ]:
list(tokenizer.word_index.items())[0:5]

[('the', 1), ('and', 2), ('to', 3), ('of', 4), ('i', 5)]

In [ ]:
# n grams for one line:
for line in text.split('\n'):
  print(line)
  print(tokenizer.texts_to_sequences([line]))
  token_list=tokenizer.texts_to_sequences([line])[0]
  print(token_list)
  print(len(token_list))
  for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        print(n_gram_sequence)
  break

[the tragedie of hamlet by william shakespeare 1599]
[[1, 687, 4, 45, 41, 1886, 1887, 1888]]
[1, 687, 4, 45, 41, 1886, 1887, 1888]
8
[1, 687]
[1, 687, 4]
[1, 687, 4, 45]
[1, 687, 4, 45, 41]
[1, 687, 4, 45, 41, 1886]
[1, 687, 4, 45, 41, 1886, 1887]
[1, 687, 4, 45, 41, 1886, 1887, 1888]


In [ ]:
## create inoput sequences
input_sequences=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
## Pad Sequences
max_sequence_len=max([len(x) for x in input_sequences])
max_sequence_len

14

In [ ]:
# padding
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))

In [ ]:
input_sequences[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,
       687], dtype=int32)

In [ ]:
##create predicitors and label
import tensorflow as tf
x,y=input_sequences[:,:-1],input_sequences[:,-1]

In [ ]:
x[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int32)

In [ ]:
y[0]

687

In [ ]:
# converting y wrt all the categories
y=tf.keras.utils.to_categorical(y,num_classes=total_words)
y[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
# Define early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
## Train our LSTM RNN

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,Input,G

## Define the model
model=Sequential()
model.add(Input(shape=(max_sequence_len-1,)))
model.add(Embedding(input_dim=total_words,output_dim=100))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation="softmax"))

# #Compile the model
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 14, 100)             │         481,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 14, 150)             │         150,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 14, 150)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 100)                 │         100,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4818)                │         486,618 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,219,418 (4.65 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
## Train the model
history=model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),verbose=1,callbacks=[early_stopping])

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.0318 - loss: 7.1222 - val_accuracy: 0.0346 - val_loss: 6.7909
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.0340 - loss: 6.4484 - val_accuracy: 0.0429 - val_loss: 6.8890
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.0438 - loss: 6.2833 - val_accuracy: 0.0470 - val_loss: 6.9186
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.0500 - loss: 6.1991 - val_accuracy: 0.0474 - val_loss: 6.9441
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.0513 - loss: 6.0767 - val_accuracy: 0.0482 - val_loss: 6.9725
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.0643 - loss: 5.8974 - val_accuracy: 0.0515 - val_loss: 7.0265


In [ ]:
# Function to predict the next word
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]  # Ensure the sequence length matches max_sequence_len-1
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [ ]:
input_text="To be or not to be"
print(f"Input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next Word PRediction:{next_word}")

Input text:To be or not to be
Next Word PRediction:the


In [ ]:
input_text="  Barn. Last night of all,When yond same"
print(f"Input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next Word PRediction:{next_word}")

Input text:  Barn. Last night of all,When yond same
Next Word PRediction:the


GRU

In [ ]:
## Define the model
model=Sequential()
model.add(Input(shape=(max_sequence_len-1,)))
model.add(Embedding(input_dim=total_words,output_dim=100))
model.add(GRU(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(100))
model.add(Dense(total_words,activation="softmax"))

# #Compile the model
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 14, 100)             │         481,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 14, 150)             │         113,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 14, 150)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 100)                 │          75,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4818)                │         486,618 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,157,418 (4.42 MB)

 Trainable params: 1,157,418 (4.42 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
## Train the model
history=model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),verbose=1) #,callbacks=[early_stopping]

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.0292 - loss: 7.1636 - val_accuracy: 0.0348 - val_loss: 6.8389
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.0402 - loss: 6.4785 - val_accuracy: 0.0459 - val_loss: 6.8952
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.0521 - loss: 6.2097 - val_accuracy: 0.0523 - val_loss: 6.8381
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.0647 - loss: 5.9745 - val_accuracy: 0.0624 - val_loss: 6.8524
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.0811 - loss: 5.7085 - val_accuracy: 0.0616 - val_loss: 6.9267
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.0907 - loss: 5.4707 - val_accuracy: 0.0653 - val_loss: 6.9592
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.0977 - loss: 5.2648 - val_accuracy: 0.0682 - val_loss: 7.0795
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.1132 - loss: 5.0180 - val_accuracy:

In [ ]:
input_text="To be or not to be"
print(f"Input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next Word PRediction:{next_word}")

Input text:To be or not to be
Next Word PRediction:as


In [ ]:
input_text="  Barn. Last night of all,When yond same"
print(f"Input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next Word PRediction:{next_word}")

Input text:  Barn. Last night of all,When yond same
Next Word PRediction:starre
